In [3]:
# Used for loding PNG
from PIL import Image
import numpy as np
import tensorflow as tf
import os
import pandas as pd
from sklearn.model_selection import train_test_split

In [4]:
def load_data(name):
    df = pd.read_csv(name + '.csv')
    df_sorted = df.sort_values(by='image_id')
    #labels = df_sorted.drop(columns=['image_id'])

    labelDict = dict()
    labels = df_sorted.values
    for entry in labels:
        try:
            labelDict[entry[0]] = entry[1]
        except:
            labelDict[entry[0]] = 0

    images = []
    labels = []
    filenames = []
    for filename in os.listdir(name):
        if filename.endswith('.png'):
            file_path = os.path.join(name, filename)
            image = Image.open(file_path)
            image_array = np.array(image)
            if filename[:-4] in labelDict.keys():
                filenames.append(filename[:-4])
                images.append(image_array)
                labels.append(labelDict[filename[:-4]])

    for i in range(len(images)):
        if images[i].shape != (80, 80, 3):
            # images[i] = np.mean(images[i], axis=2, keepdims=True)
            images[i] = np.stack((images[i], images[i], images[i]), axis=-1)
        # else:
        #     images[i] = np.reshape(images[i], (80, 80, 1))

    images = np.array(images)
    labels = np.array(labels)

    means = np.mean(images, axis=(0, 1, 2), keepdims=True)
    stds = np.std(images, axis=(0, 1, 2), keepdims=True)
    images = (images - means) / stds

    mins = np.min(images, axis=(1, 2), keepdims=True)
    maxs = np.max(images, axis=(1, 2), keepdims=True)
    images = (images - mins) / (maxs - mins) * 255.0

    return filenames, images, labels

In [5]:
#LOAD
test_filenames, test_images, test_labels = load_data('test')
train_filenames, train_images, train_labels = load_data('train')
validation_filenames, validation_images, validation_labels = load_data('validation')

train_images_subset_train, validation_images_subset_train, train_labels_subset_train, validation_labels_subset_train = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)

train_images = np.array(train_images)
validation_images = np.array(validation_images)

# Assuming train_images, train_labels, validation_images, validation_labels are already loaded
# Ensure the labels are one-hot encoded
train_labels_one_hot = tf.keras.utils.to_categorical(train_labels, num_classes=3)
validation_labels_one_hot = tf.keras.utils.to_categorical(validation_labels, num_classes=3)

train_labels_subset_train_one_hot = tf.keras.utils.to_categorical(train_labels_subset_train, num_classes=3)
validation_labels_subset_train_one_hot = tf.keras.utils.to_categorical(validation_labels_subset_train, num_classes=3)

print(np.shape(train_images))
print(np.shape(train_labels))
print(np.shape(test_images))
print(np.shape(test_labels))
print(np.shape(validation_images))
print(np.shape(validation_labels))

Table = []

(10500, 80, 80, 3)
(10500,)
(4500, 80, 80, 3)
(4500,)
(3000, 80, 80, 3)
(3000,)


In [ ]:
unique, counts = np.unique(train_labels, return_counts=True)
print(f'Training class distribution: {dict(zip(unique, counts))}')
unique, counts = np.unique(validation_labels, return_counts=True)
print(f'Validation class distribution: {dict(zip(unique, counts))}')

In [ ]:
import matplotlib.pyplot as plt

for i in range(5):
    # print(train_images[i])
    plt.imshow(train_images[i])
    plt.title(f'Training Label: {train_labels[i]}')
    plt.show()

for i in range(5):
    # print(validation_images[i])
    plt.imshow(np.uint8(validation_images[i]))
    plt.title(f'Validation Label: {validation_labels[i]}')
    plt.show()

In [ ]:
def add_gaussian_noise(image, stddev=0.1):
    noise = np.random.normal(0, stddev, image.shape)
    noisy_image = image + noise
    noisy_image = np.clip(noisy_image, 0., 1.)
    return noisy_image

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

model_train_images = train_images
model_train_labels = train_labels_one_hot
model_validation_images = validation_images
model_validation_labels = validation_labels_one_hot


# Data augmentation to prevent overfitting
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

# Fit the data generator to the training data
datagen.fit(model_train_images)

# Define the model
my_model = tf.keras.models.Sequential([
 Conv2D(16, (3,3), activation='relu' ,
 input_shape=(80, 80, 3)),
 MaxPooling2D(2, 2),
 Conv2D(32, (3,3), activation='relu'),
 MaxPooling2D(2,2),
 Conv2D(64, (3,3), activation='relu'),
 MaxPooling2D(2,2),
 Conv2D(64, (3,3), activation='relu'),
 MaxPooling2D(2,2),
 Conv2D(64, (3,3), activation='relu'),
 Flatten(),
 Dense(512, activation='relu'),
 Dense(3, activation='softmax')
])

# Compile the model
my_model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

# Early stopping callback to prevent overfitting
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with data augmentation
history = my_model.fit(
    # datagen.flow(model_train_images, model_train_labels, batch_size=32),
    model_train_images, model_train_labels,
    # batch_size=32,
    epochs=10,
    validation_data=(model_validation_images, model_validation_labels),
)

# Evaluate the model
loss, accuracy = my_model.evaluate(model_validation_images, model_validation_labels)
print(f'Validation accuracy: {accuracy}')

In [ ]:
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


model_train_images = train_images
model_train_labels = train_labels_one_hot
model_validation_images = validation_images
model_validation_labels = validation_labels_one_hot

base_model = EfficientNetB4(weights='imagenet', include_top=False, input_shape=(80, 80, 3))

# for layer in base_model.layers:
#     if isinstance(layer, tf.keras.layers.Conv2D):
#         layer.filters = int(layer.filters * 0.5)  # Reduce the number of filters by 25%

# Add custom layers on top of the modified base model
x = base_model.output
# x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
# x = Dropout(0.5)(x)

predictions = Dense(3, activation='softmax')(x)  # Assuming num_classes is the number of output classes

# Create the model
eff_net_b4_model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
eff_net_b4_model.compile(optimizer=Adam(learning_rate=2e-4), loss='categorical_crossentropy', metrics=['accuracy'])


eff_net_b4_model.fit(model_train_images, model_train_labels, epochs=5, validation_data=(model_validation_images, model_validation_labels))

In [6]:
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


model_train_images = train_images
model_train_labels = train_labels_one_hot
model_validation_images = validation_images
model_validation_labels = validation_labels_one_hot

base_model = ResNet50V2(weights='imagenet', include_top=False, input_shape=(80, 80, 3))

# for layer in base_model.layers:
#     if isinstance(layer, tf.keras.layers.Conv2D):
#         layer.filters = int(layer.filters * 0.5)  # Reduce the number of filters by 25%
# for layer in base_model.layers:
#     if isinstance(layer, tf.keras.layers.Conv2D):
#         layer.filters = int(layer.filters * 0.5)

# num_blocks_to_remove = 5  # Example: Remove the last 5 blocks
# for _ in range(num_blocks_to_remove):
#     base_model.layers.pop()

# Add custom layers on top of the modified base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
# x = Dropout(0.5)(x)

predictions = Dense(3, activation='softmax')(x)  # Assuming num_classes is the number of output classes

# Create the model
res_net_model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
res_net_model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])


res_net_model.fit(model_train_images, model_train_labels, epochs=5, validation_data=(model_validation_images, model_validation_labels))

2024-05-28 21:23:00.479430: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_COMPAT_NOT_SUPPORTED_ON_DEVICE: forward compatibility was attempted on non supported HW
2024-05-28 21:23:00.479463: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:134] retrieving CUDA diagnostic information for host: eduard-IdeaPad-5-Pro-16ACH6
2024-05-28 21:23:00.479471: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:141] hostname: eduard-IdeaPad-5-Pro-16ACH6
2024-05-28 21:23:00.479653: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:165] libcuda reported version is: 555.42.2
2024-05-28 21:23:00.479676: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:169] kernel reported version is: 535.171.4
2024-05-28 21:23:00.479682: E external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:251] kernel version 535.171.4 does not match DSO version 555.42.2 -- cannot find working devices in thi

Epoch 1/5


2024-05-28 21:23:01.620924: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 806400000 exceeds 10% of free system memory.


329/329 ━━━━━━━━━━━━━━━━━━━━ 0s 961ms/step - accuracy: 0.4331 - loss: 1.5224

2024-05-28 21:28:35.846874: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 230400000 exceeds 10% of free system memory.


329/329 ━━━━━━━━━━━━━━━━━━━━ 345s 997ms/step - accuracy: 0.4332 - loss: 1.5216 - val_accuracy: 0.5493 - val_loss: 0.9720
Epoch 2/5
329/329 ━━━━━━━━━━━━━━━━━━━━ 333s 1s/step - accuracy: 0.5986 - loss: 0.8645 - val_accuracy: 0.6097 - val_loss: 0.8746
Epoch 3/5
329/329 ━━━━━━━━━━━━━━━━━━━━ 334s 1s/step - accuracy: 0.7075 - loss: 0.6799 - val_accuracy: 0.6193 - val_loss: 0.8781
Epoch 4/5
329/329 ━━━━━━━━━━━━━━━━━━━━ 354s 1s/step - accuracy: 0.7861 - loss: 0.5131 - val_accuracy: 0.5767 - val_loss: 1.0221
Epoch 5/5
329/329 ━━━━━━━━━━━━━━━━━━━━ 385s 1s/step - accuracy: 0.8304 - loss: 0.4059 - val_accuracy: 0.6110 - val_loss: 1.0279


In [ ]:
from sklearn.metrics import confusion_matrix

model = base_model

y_prediction = model.predict(validation_images)
y_prediction = np.argmax (y_prediction, axis = 1)
# y_test=np.argmax(validation_labels, axis=1)
#Create confusion matrix and normalizes it over predicted (columns)
result = confusion_matrix(validation_labels, y_prediction , normalize='pred')
print(result)

2024-05-28 22:01:21.547522: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 230400000 exceeds 10% of free system memory.


36/94 ━━━━━━━━━━━━━━━━━━━━ 7s 124ms/step

In [ ]:
import matplotlib.pyplot as plt

# history = model.fit(
#     datagen.flow(train_images, train_labels_one_hot),
#     epochs=50,
#     validation_data=(validation_images, validation_labels_one_hot),
#     callbacks=[early_stopping]
# )

plt.plot(history.history['accuracy'], label='train_accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()
plt.show()

In [ ]:

validation_labels_one_hot = tf.keras.utils.to_categorical(validation_labels, num_classes=3)

model.evaluate(validation_images, validation_labels_one_hot)
model.evaluate(train_images, train_labels_one_hot)
predict = model.predict(validation_images)

print(validation_labels_one_hot)

for i in range(len(validation_labels_one_hot)):
    print(f'{validation_labels_one_hot[i]} - {np.round(predict[i])}')


In [ ]:
def printPrediction(filenames, prediction, name):
    d = dict()
    for i in range(len(filenames)):
        d[filenames[i]] = prediction[i]


    df = pd.read_csv('sample_submission.csv')
    for i in range(len(filenames)):
        df.at[i, 'label'] = d[df.at[i, 'image_id']]
    df.to_csv(name, index=False)

prediction = my_model.predict(test_images)
prediction = np.argmax (prediction, axis = 1)

print(*zip(prediction, test_filenames), sep='\n')

printPrediction(test_filenames, prediction, 'cnn.csv')